# Assignment 4

Welcome to Assignment 4. This will be the most fun. Now we will prepare data for plotting.

Just make sure you hit the play button on each cell from top to down. There are three functions you have to implement. Please also make sure than on each change on a function you hit the play button again on the corresponding cell to make it available to the rest of this notebook. Please also make sure to only implement the function bodies and DON'T add any additional code outside functions since this might confuse the autograder.

So the function below is used to make it easy for you to create a data frame from a cloudant data frame using the so called "DataSource" which is some sort of a plugin which allows ApacheSpark to use different data sources.


In [11]:
#Please don't modify this function
def readDataFrameFromCloudant(host,user,pw,database):
    cloudantdata=spark.read.format("com.cloudant.spark"). \
    option("cloudant.host",host). \
    option("cloudant.username", user). \
    option("cloudant.password", pw). \
    load(database)

    cloudantdata.createOrReplaceTempView("washing")
    spark.sql("SELECT * from washing").show()
    return cloudantdata

Sampling is one of the most important things when it comes to visualization because often the data set get so huge that you simply

- can't copy all data to a local Spark driver (Data Science Experience is using a "local" Spark driver)
- can't throw all data at the plotting library

Please implement a function which returns a 10% sample of a given data frame:

In [12]:
def getSample(df,spark):
    
    result = spark.sql("select d.temperature from washing where d.temperature is not null")
    
    mySearchingPiece = int((result.count()*10)/100)
    
    # result_array = result.rdd.map(lambda row : row.temperature).sample(False,0.1)
    result_array = result.rdd.map(lambda row : row.temperature).sample(False,mySearchingPiece)
    return result_array

Now we want to create a histogram and boxplot. Please ignore the sampling for now and retur a python list containing all temperature values from the data set

In [13]:
def getListForHistogramAndBoxPlot(df,spark):
    
    result = spark.sql("select d.temperature from washing where d.temperature is not null")
    
    mySearchingPiece = int((result.count()*10)/100)
    
    # result_array = result.rdd.map(lambda row : row.temperature).sample(False,0.1)
    result_array = result.rdd.map(lambda row : row.temperature).sample(False,mySearchingPiece).collect()
           
    return result_array

Finally we want to create a run chart. Please return two lists (encapusalted in a python tuple object) containing temperature and timestamp (ts) ordered by timestamp. Please refere to the following link to learn more about tuples in python: https://www.tutorialspoint.com/python/python_tuples.htm

In [14]:
#should return a tuple containing the two lists for timestamp and temperature
#please make sure you take only 10% of the data by sampling
#please also ensure that you sample in a way that the timestamp samples and temperature samples correspond (=> call sample on an object still containing both dimensions)
def getListsForRunChart(df,spark):
    
    result = spark.sql("select d.temperature,d.ts from washing where d.temperature is not null order by d.ts asc")
    
    mySearchingPiece = int((result.count()*10)/100)
    
    # result_rdd = result.rdd.sample(False,0.1).map(lambda row : (row.ts,row.temperature))
    result_rdd = result.rdd.sample(False,mySearchingPiece).map(lambda row : (row.ts,row.temperature))
    
    result_array_ts = result_rdd.map(lambda (ts,temperature): ts).collect()
    result_array_temperature = result_rdd.map(lambda (ts,temperature): temperature).collect()
    return (result_array_ts,result_array_temperature)

### PLEASE DON'T REMOVE THIS BLOCK - THE FOLLOWING CODE IS NOT GRADED
#axx
### PLEASE DON'T REMOVE THIS BLOCK - THE FOLLOWING CODE IS NOT GRADED

In [15]:
#TODO Please provide your Cloudant credentials here
hostname = "0f8410a3-0e0f-467f-a3cf-b9fbd5da37c2-bluemix.cloudant.com"
user = "0f8410a3-0e0f-467f-a3cf-b9fbd5da37c2-bluemix"
pw = "1363e31faf953220b28f78a9afae906603f2882db47b0e9375bf650569f4d368"
database = "washing"
cloudantdata=readDataFrameFromCloudant(hostname, user, pw, database)

+--------------------+--------------------+--------------------+
|                 _id|                _rev|                   d|
+--------------------+--------------------+--------------------+
|0f83931363c7ce12d...|1-ffedf0982164b41...|[2,null,null,74,n...|
|0f83931363c7ce12d...|1-25fb1d3ec6eecb7...|[2,null,null,null...|
|0f83931363c7ce12d...|1-69c8b2d638db12e...|[10,11,acceptable...|
|0f83931363c7ce12d...|1-2e2a41b45fcb93a...|[11,11,acceptable...|
|0f83931363c7ce12d...|1-7f45175964250f8...|[5,null,null,68,n...|
|0f83931363c7ce12d...|1-75f7f3cdb9a3787...|[17,11,acceptable...|
|0f83931363c7ce12d...|1-a53fb4cd7965f0f...|[20,11,acceptable...|
|0f83931363c7ce12d...|1-2507d6eb21e3936...|[34,11,acceptable...|
|0f83931363c7ce12d...|1-fc51c2dbb04a718...|[43,11,acceptable...|
|0f83931363c7ce12d...|1-91dd8f0ce02523b...|[50,11,acceptable...|
|0f83931363c7ce12d...|1-3685ecced4c3b9f...|[51,11,acceptable...|
|0f83931363c7ce12d...|1-71d10c74297acf1...|[55,11,acceptable...|
|0f83931363c7ce12d...|1-c

In [16]:
%matplotlib inline
import matplotlib.pyplot as plt

In [17]:
plt.hist(getListForHistogramAndBoxPlot(cloudantdata,spark))
plt.show()

AnalysisException: u"cannot resolve '`temperature`' given input columns: [_id, _rev, d]; line 1 pos 38;\n'Project ['temperature]\n+- 'Filter isnotnull('temperature)\n   +- SubqueryAlias washing\n      +- Relation[_id#7,_rev#8,d#9] CloudantReadWriteRelation(com.cloudant.spark.CloudantConfig@fcfb6009,StructType(StructField(_id,StringType,true), StructField(_rev,StringType,true), StructField(d,StructType(StructField(count,LongType,true), StructField(flowrate,LongType,true), StructField(fluidlevel,StringType,true), StructField(frequency,LongType,true), StructField(hardness,LongType,true), StructField(speed,LongType,true), StructField(temperature,LongType,true), StructField(ts,LongType,true), StructField(voltage,LongType,true)),true)),[_id: string, _rev: string ... 1 more field])\n"

In [ ]:
plt.boxplot(getListForHistogramAndBoxPlot(cloudantdata,spark))
plt.show()

In [ ]:
lists = getListsForRunChart(cloudantdata,spark)

In [ ]:
plt.plot(lists[0],lists[1])
plt.xlabel("time")
plt.ylabel("temperature")
plt.show()

Congratulations, you are done! Please download the notebook as python file, name it assignment4.1.py and sumbit it to the grader.